<a href="https://colab.research.google.com/github/fritzmartin003/RAG-System-Projekt/blob/main/Sara%20Test%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notwendige Bibliotheken installieren
!pip install faiss-cpu transformers sentence-transformers pymupdf numpy scipy
!pip uninstall -y tensorflow


In [2]:
# Imports
import fitz  # PyMuPDF
import numpy as np
import faiss
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
import sklearn
import torch
import time

In [3]:
from google.colab import userdata
HuggingFaceAPIKey = "HF2"

In [5]:
# PDF-Text extrahieren
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text") + "\n"
    return text

pdf_path = "SakowskiBuch.pdf"
pdf_text = extract_text_from_pdf(pdf_path)


In [22]:
# Text in Chunks teilen
def split_text(text, chunk_size=500, overlap=90):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = split_text(pdf_text)
print(f" PDF in {len(chunks)} Chunks unterteilt!")

 PDF in 1521 Chunks unterteilt!


In [ ]:
# Embedding Modell laden
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Optimale Batch-Größe finden (Experimentieren!)
optimal_batch_size = 3000  # Starte mit einem hohen Wert und reduziere, wenn du Speicherprobleme hast

def create_faiss_index(chunks):
    start_time = time.time()  # Zeitmessung

    # Embeddings erstellen (mit optimierter Batch-Verarbeitung und Tensor-Konvertierung)
    chunk_embeddings_tensor = embedding_model.encode(chunks, batch_size=optimal_batch_size, convert_to_tensor=True)
    chunk_embeddings = chunk_embeddings_tensor.cpu().numpy()

    # Normalisieren der Embeddings (wichtig für die meisten Distanzmetriken)
    chunk_embeddings = sklearn.preprocessing.normalize(chunk_embeddings)

    dimension = chunk_embeddings.shape[1]

    # FAISS Index erstellen (IndexIVFFlat mit Training)
    nlist = int(np.sqrt(len(chunk_embeddings)))  # Anzahl der Partitionen (Faustregel)
    quantizer = faiss.IndexFlatL2(dimension)  # Quantisierer für die Clusterzentren
    index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)

    # Training des Index (NOTWENDIG für IndexIVFFlat!)
    index.train(chunk_embeddings)

    # Hinzufügen der Embeddings zum Index (NACH dem Training)
    index.add(chunk_embeddings)

    end_time = time.time()
    print(f"FAISS Vektordatenbank erstellt! (Zeit: {end_time - start_time:.2f} Sekunden)")
    return index

index = create_faiss_index(chunks)

In [9]:
def search(index, query, k=5):
    query_embedding_tensor = embedding_model.encode([query], convert_to_tensor=True)
    query_embedding = query_embedding_tensor.cpu().numpy()

    query_embedding = sklearn.preprocessing.normalize(query_embedding)
    D, I = index.search(query_embedding, k)  # D: Distanzen, I: Indizes
    return D, I

def get_search_results(index, query, chunks, k=5):
    D, I = search(index, query, k)
    results = []
    for i, distance in zip(I[0], D[0]):
        results.append({"chunk": chunks[i], "distance": distance})
    return results


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import os
import torch

!huggingface-cli login

os.environ["HUGGINGFACE_API_KEY"] = "HF2"
model_name = "mistralai/Mistral-7B-v0.1"  # Alternativ: "meta-llama/Llama-2-7b-chat-hf"

# Tokenizer & Modell mit explizitem Token laden
pipe = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

def generate_answer(query):
    relevant_chunks = get_search_results(index, query, chunks, k=10)
    for relevant_chunk in relevant_chunks:
        print(f"Chunk: {relevant_chunk['chunk']}")
        print(f"Distanz: {relevant_chunk['distance']:.4f}")
        print("-" * 20)
    # Extract the text content from the relevant_chunks list of dictionaries
    context = "\n".join([chunk['chunk'] for chunk in relevant_chunks])
    prompt = f"Beantworte die Frage basierend auf diesem Kontext:\n\n{context}\n\nFrage: {query}\nAntwort:"

    response = pipe(prompt, max_new_tokens=128, do_sample=True, temperature=0.5)
    return response[0]["generated_text"]

In [ ]:
# Test
frage = "Welche Besonderheiten bestehen für Auszubildende?"
antwort = generate_answer(frage)
print("Antwort:", antwort)